#Load data

In [ ]:
# Mount Google Drive locally 
# Using the instructions found here https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA&line=1&uniqifier=1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the pickled dataframe using the "read_pickle()"" function from "pandas"
# df = ...

# Classify outcomes based on applicant income

In [ ]:
import numpy as np

df[['log(applicant_income_000s)']] = np.log(df['applicant_income_000s'].copy())
#pred_35 = ...
#pred_43 = ...
#pred_50 = ...

assert(pred_35.sum() == 140608)
assert(pred_43.sum() == 70354)
assert(pred_50.sum() == 19405)

# Compute and chart classifier metrics

In [ ]:
def accept_rate_group(group=None):
  def accept_rate(pred):
    def pred_group():
      return pred[group] if group is not None else pred
    return pred_group().mean() * 100
  return accept_rate

female = (df['applicant_sex_name'] == "Female").copy()
male = (df['applicant_sex_name'] == "Male").copy()

accept_rate = accept_rate_group()
accept_rate_male = accept_rate_group(male)
accept_rate_female = accept_rate_group(female)

## acceptance rate % across thresholds
### total
# accept_rate_35 = ...
# accept_rate_43 = ...
# accept_rate_50 = ...

### male
# accept_rate_male_35 = ...
# accept_rate_male_43 = ...
# accept_rate_male_50 = ...

### female
# accept_rate_female_35 = ... 
# accept_rate_female_43 = ...
# accept_rate_female_50 = ... 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# chart different thresholds: acceptance rates, disparate impact and 2-SD rule
d = pd.DataFrame({"Threshold": [3.5, 4.3, 5.0],
                  "Total Acceptance Rate": [accept_rate_35, accept_rate_43, accept_rate_50],
                  "Female Acceptance Rate": [accept_rate_female_35, accept_rate_female_43, accept_rate_female_50],
                  "Male Acceptance Rate": [accept_rate_male_35, accept_rate_male_43, accept_rate_male_50]})

# sns.barplot(...) Total Acceptance Rate
plt.show()

In [ ]:
# sns.barplot(...) Female Acceptance Rate
plt.show()

In [ ]:
# sns.barplot(...) Male Acceptance Rate
plt.show()

# Compute and chart the Disparate Impact metric

In [ ]:
def DispImpact(pred, group_a, group_b):
  ### Disparate Impact
  SR_A = pred[group_a==1].mean() # success rate group A
  SR_B = pred[group_b==1].mean() # success rate group B
  return SR_A/SR_B

## disparate impact across thresholds
# DP_35 = ...
# DP_43 = ...
# DP_50 = ...

from math import isclose

assert(isclose(DP_35, 0.864, rel_tol=0.001))
assert(isclose(DP_43, 0.605, rel_tol=0.001))
assert(isclose(DP_50, 0.3996, rel_tol=0.001))

d = pd.DataFrame({"Threshold": [3.5, 4.3, 5.0],
                  "Disparate Impact": [DP_35, DP_43, DP_50]})

# sns.barplot(...) Disparate Impact
plt.show()

# Compute and chart the 2-SD Rule Metric

In [ ]:
def TwoSDRule(pred, group_a, group_b):
    import numpy as np

    ### 2-SD Rule
    SR_A = pred[group_a==1].mean() # success rate group A
    SR_B = pred[group_b==1].mean() # success rate group B
    SR_T = pred.mean() # success rate total
    P_min = (group_a==1).mean() # minority proportion - usually group A
    N = len(pred) # sample size
    return (SR_A - SR_B)/np.sqrt( (SR_T * (1.0 - SR_T))/(N * P_min * (1 - P_min)))

## 2-SD rule across thresholds
# Z_35 = ...
# Z_43 = ...
# Z_50 = ...

assert(isclose(Z_35, -62.3, rel_tol=0.001))
assert(isclose(Z_43, -72.3, rel_tol=0.001))
assert(isclose(Z_50, -50.2, rel_tol=0.001))

d = pd.DataFrame({"Threshold": [3.5, 4.3, 5.0],
                  "2-SD Rule": [Z_35, Z_43, Z_50]})

# sns.barplot(...) 2-SD Rule
plt.show()